# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create queries using Apache Cassandra

## The CSV file titled *event_datafile_new.csv*, contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

## Query 1: 

### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Query 1 expects the name of the artist, title of the song, and length of the track based on sessionID and itemInSession. 

The query to get the data will be a SELECT statement like: 

`SELECT name of the artist, title of the song, length of the track FROM TABLE_NAME WHERE sessionId = value AND itemInSession = value`

Based on this SELECT query, the CREATE table query will look like the following:

`Table Name: music_library_by_item
column 1: Session ID
column 2: Item In Session
column 3: Artist Name
column 4: Song Title
column 5: Song Length
PRIMARY KEY(Session ID, Item In Session)`

NOT EXIST will be added to the CREATE statement to check if the table exists and only creates the table if it does not exist. The primary key contains both sessionId and itemInSession to uniquely identify each row in the table.

In [11]:
# Create table for Query 1
query = "CREATE TABLE IF NOT EXISTS music_library_by_item "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, \
                  PRIMARY KEY (session_id, item_in_session))" 

session.execute(query)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_by_item (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [13]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, song_length FROM music_library_by_item WHERE session_id=338 AND item_in_session=4"

# Execute query
rows = session.execute(query)

# Convert to pandas dataframe
df = pd.DataFrame(data=list(rows))

# Show only the following columns of dataframe
df[['artist_name','song_title','song_length']]

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2: 

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182

Query 2 expects the name of the artist, title of the song sorted by itemInSession, and the user's first and last names based on userId and sessionId. 

The query to get the data will be a SELECT statement like: 

`SELECT name of the artist, title of the song, first name of the user, last name of the user FROM TABLE_NAME WHERE userId = value AND sessionId = value`

Based on this SELECT query, the CREATE table query will look like the following:

`Table Name: music_library_by_user
column 1: User ID
column 2: Session ID
column 3: Item In Session
column 4: Artist Name
column 5: Song Title
column 6: User First Name
column 7: User Last Name
PRIMARY KEY((User ID, Session ID), Item In Session))`

NOT EXIST will be added to the CREATE statement to check if the table exists and only creates the table if it does not exist. The primary key contains both user ID and itemInSession to uniquely identify each row in the table. Additionally, the primary key includes a clustering column, itemInSession, to sort the songs by itemInSession in ascending order.

In [14]:
# Create table for Query 2
query = "CREATE TABLE IF NOT EXISTS music_library_by_user "
query = query + "(user_id int, session_id int, item_in_session int, \
                  artist_name text, song_title text, user_firstname text, user_lastname text, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))" 

session.execute(query)                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_by_user (user_id, session_id, item_in_session, \
                 artist_name, song_title, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 

In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, user_firstname, user_lastname FROM music_library_by_user WHERE user_id=10 AND session_id=182"

# Execute query
rows = session.execute(query)

# Convert to pandas dataframe
df = pd.DataFrame(data=list(rows))

# Show only the following columns of dataframe
df[['artist_name','song_title', 'user_firstname','user_lastname']]

,artist_name,song_title,user_firstname,user_lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3: 

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Query 3 expects the user's first and last names based on the title of the song. 

The query to get the data will be a SELECT statement like: 

`SELECT first name of the user, last name of the user FROM TABLE_NAME WHERE song title = value`

Based on this SELECT query, the CREATE table query will look like the following:

`Table Name: users_to_songs
column 1: Song Title
column 2: User ID
column 3: User First Name
column 4: User Last Name
PRIMARY KEY(Song Title, User ID)`

NOT EXIST will be added to the CREATE statement to check if the table exists and only creates the table if it does not exist. The primary key contains both the title of the song and the user's ID to uniquely identify each row in the table. Using both the user's first and last names as part of the primary key may not be unique enough. The user's ID, however, should be unique for each user.

In [17]:
# Create table for Query 3
query = "CREATE TABLE IF NOT EXISTS users_to_songs "
query = query + "(song_title text, user_id int, user_firstname text, user_lastname text, \
                  PRIMARY KEY (song_title, user_id))" 

session.execute(query)                   

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_to_songs (song_title, user_id, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) 

In [20]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT user_firstname, user_lastname FROM users_to_songs WHERE song_title='All Hands Against His Own'"

# Execute query
rows = session.execute(query)

# Convert to pandas dataframe
df = pd.DataFrame(data=list(rows))

# Show only the following columns of dataframe
df[['user_firstname','user_lastname']]

,user_firstname,user_lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE music_library_by_item"
session.execute(query)

In [18]:
query = "DROP TABLE music_library_by_user"
session.execute(query)

In [19]:
query = "DROP TABLE users_to_songs"
session.execute(query)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()

In [21]:
# list of packages and versions 
!pip freeze

altair==1.2.1
asn1crypto==0.22.0
atari-py==0.1.7
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.6
av==0.3.3
awscli==1.16.17
backcall==0.1.0
backports.functools-lru-cache==1.4
backports.weakref==1.0rc1
beautifulsoup4==4.6.0
bleach==1.5.0
blinker==1.4
bokeh==0.12.13
boto==2.48.0
boto3==1.9.7
botocore==1.12.7
Box2D==2.3.2
Box2D-kengz==2.3.3
bresenham==0.2
bz2file==0.98
cassandra-driver==3.11.0
certifi==2019.6.16
cffi==1.11.2
chardet==3.0.4
click==6.7
cloudpickle==0.2.2
colorama==0.3.9
colour==0.1.5
conda==4.6.14
cryptography==2.1.4
cycler==0.10.0
Cython==0.29.7
cytoolz==0.9.0.1
dask==0.16.1
decorator==4.0.11
defusedxml==0.5.0
dill==0.2.7.1
docutils==0.14
dronekit==2.9.2
entrypoints==0.2.3
et-xmlfile==1.0.1
eventlet==0.22.0
fastcache==1.0.2
Flask==0.12.2
Flask-SocketIO==2.9.2
future==0.16.0
gensim==3.4.0
glfw==1.8.0
greenlet==0.4.12
gym==0.9.6
h5py==2.7.1
heapdict==1.0.0
html2text==2018.1.9
html5lib==0.9999999
idna==2.6
imageio==2.1.2
ipykernel==4.9.0
ipython==6.5.0
ipython-genutils==0.2

In [22]:
!python --version

Python 3.6.3
